In [16]:
import sqlite3 as sq
import numpy as np
import pickle
from scipy.optimize import curve_fit
from scipy.stats import linregress
from matplotlib import pyplot as plt
from multiprocessing import pool
%matplotlib inline

In [7]:
def get_table(conn, table):
    out = None
    with conn:
        out = conn.execute(f"select * from {table};").fetchall();
    out_dict = {k[0]: np.array(k[1:]) for k in out}
    return (get_tau(out), out_dict)

def get_tau(table):
    return np.array(range(1, len(table[0]))) if len(table) >= 1 else None

def plot(data, key, q):
    plt.plot(data[key][0], data[key][1][q])

In [8]:
conn = sq.connect("../crowd.sqlite")

In [9]:
with conn:
    tables = conn.execute("SELECT name FROM main.sqlite_master WHERE type='table'").fetchall();

In [10]:
tables_filtered = filter(lambda t: t.find("video_multiDDM") != -1 and t.find("1024") != -1, map(lambda t: t[0], tables))

In [11]:
singleDDM_data = {k: get_table(conn, k) for k in tables_filtered}

In [12]:
crowd_data = {k: v for k, v in singleDDM_data.items() if k.find("crowd") != -1}
running_data = {k: v for k, v in singleDDM_data.items() if k.find("running") != -1}

In [13]:
def brownian_isf(tau, tauc, a, b):
    return a * ( 1 - np.exp(-tau / tauc)) + b
def ballistic_isf(tau, v, tauc, a, b):
    return a * (1 - np.sinc(v * tau) * np.exp(- tau / tauc)) + b

In [21]:
def fit(f, x, y, bounds=([0, -np.inf, -np.inf], [np.inf, np.inf, np.inf])):
    try:
        params, errs = curve_fit(f, x, y, bounds=bounds)
        return params, np.sqrt(np.diag(errs))
    except RuntimeError:
        return None
def total_fit(data, f, bounds = ([0, -np.inf, -np.inf], [np.inf, np.inf, np.inf])):
    #do pool map here
    pool = pool.Pool(5)
    
    return {k: [(q, fit(f, v[0], d, bounds = bounds)) for q, d in v[1].items()] for k, v in data.items()}

In [15]:
fit_crowd_data = total_fit(crowd_data, brownian_isf)

In [20]:
with open("fit_crowd_single_ddm.pickle", "wb") as f:
    pickle.dump(fit_crowd_data, f)

In [ ]:
bounds = ([-np.inf, 0, -np.inf, -np.inf], [np.inf, np.inf, np.inf, np.inf])
fit_running_data = total_fit(running_data, ballistic_isf, bounds)

In [ ]:
with open("fit_running_single_ddm.pickle", "wb") as f:
    pickle.dump(fit_running_data, f)